先存一个网址                                       **[caffe中文社区链接](http://www.caffecn.cn/)**

# 1.caffe solver
- caffe solver（求解器）是caffe核心中的核心，在里面定义了整个训练的流程，包括调用设计好的模型，设定训练参数，测试等，具体的执行细节在他的各个子参数中
- [这里是写的比较好的caffe介绍的系列博客](http://www.cnblogs.com/denny402/default.html?page=2)

# 2.caffe 模型定义  
>1.google protobuf介绍  
>- [protobuf document](https://developers.google.com/protocol-buffers/)
>- [protobuf github adress](https://github.com/google/protobuf)  
 
>2.caffe.proto
- 这里面提供了prototxt的编写范例，里面有各种参数的说明
- **LeNet定义**
    * 首先是数据层type=data  
    * transform_param中scale定义的0.00390625=1/256
    * 在数据层中，两个top后跟lable和data，表示最终产生两个blob（data和lable）
    * **blob是什么？**
        * binary large object（二进制大对象）是一个可以存储二进制文件的容器。由于尺寸大，必须使用特殊的方式来处理。例如：上传，下载或存放到数据库。
        * 疑问：data blob是怎么组织的，才能保证下一层可以正常读取，约定是什么
    * 其次是第一层卷积层
        * 疑问：该层中lr_mult和solver.protxt中lr的联系和区别？
        * 疑问：***xavier方法是什么？***
    * 依次定义好卷积层，池化层
    * 然后定义全连接层
        * 在caffe里面被称为innerProduct
        * 疑问：为什么只有在全连接层才使用ReLu激活函数，前面为什么不用？
    * 在定义好了全连接层及ReLu层后，再添加一层全连接层
    * softmax_loss层
        * 定义好了softmax和multinomial logistic loss
        * 常见损失函数有
            1. 0-1损失函数
            2. 平方损失函数
            3. 绝对值损失函数
            4. 对数损失函数
        * 在逻辑回归中采用的是对数损失函数
        * 逻辑回归推导引申出的问题
            1. 伯努利分布（0-1分布）定义？
            2. [logistic回归详解：损失函数](http://blog.csdn.net/bitcarmanlee/article/details/51165444)
            3. [几种常见分布的介绍](http://blog.csdn.net/michael_r_chang/article/details/39188321)
            4. 最终的对数损失函数形式如下  
                * $$cost(h_{\theta}(x),y) = \sum_{i=1}^{m} -y_ilog(h_{\theta}(x)) - (1-y_i)log(1-h_{\theta}(x))$$
            5. 疑问：为什么要使用对数损失函数？有什么优势？二元logistic回归是上面形式，多元logistic回归是什么形式？
            
# 3.模型训练
>#!/usr/bin/env sh
set -e

>./build/tools/caffe train --solver=examples/mnist/lenet_solver.prototxt $@ 


1. $ $$#列出shell脚本的参数个数

2. $ $$@列出了shell脚本的参数内容  

---  
$
---
# 4.caffe 架构流程的四个层级(blob,layer,net,solver)

### 1.Blob
>Blob是caffe基本的数据结构，用四维矩阵 Batch×Channel×Height×Weight表示，存储了网络的神经元激活值和网络参数  

### 2.Layer
>Layer代表了神经网络中各种各样的层，组合成一个网络。一般一个图像或样本会从数据层中读进来，然后一层一层的往后传

### 3.Net
>Net就是把各种层按train_val.prototxt的定义堆叠在一起，首先进行每个层的初始化，然后不断进行Update，每更新一次就进行一次整体的前向传播和反向传播，然后把每层计算得到的梯度计算进去，完成一次更新，这里注意每层在Backward中只是计算dW和db，而W和b的更新是在Net的Update里最后一起更新的。而且在caffe里训练模型的时候一般会有两个Net，一个train一个test。刚开始训练网络时前面的一大堆输出，网络的结构什么的也都是这里输出的。

### 4.Solver
>Solver是按solver.prototxt的参数定义对Net进行训练，首先会初始化一个TrainNet和一个TestNet，然后其中的Step函数会对网络不断进行迭代，主要就是两个步骤反复迭代：①不断利用ComputeUpdateValue计算迭代相关参数，比如计算learning rate，把weight decay加上什么的，②调用Net的Update函数对整个网络进行更新。迭代中的一大堆输出也是在这里输出的，比如当前的loss和learning rate什么的。

# 5.caffe模型训练的入口函数train解析

